In [ ]:
!pip uninstall numpy
!pip install numpy
!pip install --force-reinstall catboost

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires 

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import PredefinedHoldoutSplit
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report

#### From our experiments, we found that most models achieve better performance when using LASSO-selected features with standardization, while excluding the "Entrance Test" feature. Therefore, we will proceed with the remaining experiments exclusively on the "New_Train_Features_Selected" dataset.

In [ ]:
%cd /content

/content


In [ ]:
y = pd.read_csv('Train_Target.csv').iloc[:,1]
X = pd.read_csv('Train_Features.csv').iloc[:,1:]
X_test = pd.read_csv('Test_Features.csv').iloc[:,1:]

# Preprocessing

In [ ]:
X[X.isna().any(axis = 1)].head()

,Gender,Percent_SSC,Board_SSC,Percent_HSC,Board_HSC,Stream_HSC,Percent_Degree,Course_Degree,Experience_Yrs,Entrance_Test,S-TEST,Percentile_ET,S-TEST*SCORE,Percent_MBA,Specialization_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA
0,M,56.0,ICSE,58.0,ISC,Commerce,67.00,Management,0,NaN,0,0.0,0.0,65.28,Marketing & Finance,62,77,77
17,M,52.5,Others,53.4,Others,Commerce,55.21,Management,1,NaN,0,0.0,0.0,72.00,Marketing & Finance,73,79,55
26,M,56.0,Others,42.0,ISC,Science,59.00,Computer Applications,2,NaN,0,0.0,0.0,51.48,Marketing & Finance,50,53,55
35,M,58.0,Others,54.5,Others,Commerce,49.85,Commerce,0,NaN,0,0.0,0.0,62.89,Marketing & Finance,55,67,73
37,M,58.0,ICSE,59.0,ISC,Commerce,56.00,Management,0,NaN,0,0.0,0.0,55.10,Marketing & Finance,50,57,50


In [ ]:
mode_test = X['Entrance_Test'].mode()[0]
X.fillna({'Entrance_Test':mode_test}, inplace = True)

mode_X_test = X_test['Entrance_Test'].mode()[0]
X_test.fillna({'Entrance_Test':mode_X_test}, inplace = True)

In [ ]:
print(X[X.isna().any(axis = 1)])
print(X_test[X_test.isna().any(axis = 1)])

Empty DataFrame
Columns: [Gender, Percent_SSC, Board_SSC, Percent_HSC, Board_HSC, Stream_HSC, Percent_Degree, Course_Degree, Experience_Yrs, Entrance_Test, S-TEST, Percentile_ET, S-TEST*SCORE, Percent_MBA, Specialization_MBA, Marks_Communication, Marks_Projectwork, Marks_BOCA]
Index: []
Empty DataFrame
Columns: [Gender, Percent_SSC, Board_SSC, Percent_HSC, Board_HSC, Stream_HSC, Percent_Degree, Course_Degree, Experience_Yrs, Entrance_Test, S-TEST, Percentile_ET, S-TEST*SCORE, Percent_MBA, Specialization_MBA, Marks_Communication, Marks_Projectwork, Marks_BOCA]
Index: []


In [ ]:
X['Gender'] = X['Gender'].map({'M':1, 'F': 0})
X_test['Gender'] = X_test['Gender'].map({'M':1, 'F': 0})

X_encoded = X
X_test_encoded = X_test

for col in {'Board_SSC', 'Board_HSC', 'Stream_HSC', 'Course_Degree', 'Entrance_Test', 'Specialization_MBA'}:
    X_encoded = pd.get_dummies(X_encoded, columns = [col])

for col in X_encoded.columns:
    if X_encoded[col].dtype == bool:
        X_encoded[col] = X_encoded[col].map({True: 1, False: 0})

for col in {'Board_SSC', 'Board_HSC', 'Stream_HSC', 'Course_Degree', 'Entrance_Test', 'Specialization_MBA'}:
    X_test_encoded = pd.get_dummies(X_test_encoded, columns = [col])

for col in X_test_encoded.columns:
    if X_test_encoded[col].dtype == bool:
        X_test_encoded[col] = X_test_encoded[col].map({True: 1, False: 0})

In [ ]:
from sklearn.preprocessing import Normalizer

X_normalized = X_encoded.copy()
X_test_normalized = X_test_encoded.copy()

scaler = Normalizer()
numerical_columns = ['Percent_SSC', 'Percent_HSC', 'Percent_Degree', 'Experience_Yrs','S-TEST', 'Percentile_ET', 'S-TEST*SCORE','Percent_MBA', 'Marks_Communication', 'Marks_Projectwork', 'Marks_BOCA']
X_normalized[numerical_columns] = scaler.fit_transform(X_encoded[numerical_columns])
X_test_normalized[numerical_columns] = scaler.fit_transform(X_test_encoded[numerical_columns])

In [ ]:
X_full = X_normalized.drop(X_normalized.columns[[26, 27,28]], axis = 1)
X_full_test = X_test_normalized.drop(X_test_normalized.columns[[29]], axis = 1)

X_numerical = X_normalized.iloc[:,[0,1,2,3,4]]
X_numerical_test = X_test_normalized.iloc[:,[0,1,2,3,4]]

# Train - Test Split

In [ ]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [ ]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [ ]:
X_train_num, X_temp_num, y_train_num, y_temp_num = train_test_split(X_numerical, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_num, X_test_num, y_val_num, y_test_num = train_test_split(X_temp_num, y_temp_num, test_size = 0.5, random_state = 2025, stratify = y_temp_num)

In [ ]:
np.random.seed(2025)
train_ind_num, val_ind_num = train_test_split(np.arange(X_temp_num.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_num)
split_num = PredefinedHoldoutSplit(valid_indices = val_ind_num)

# KNN

In [ ]:
params = {
    'n_neighbors': Integer(1, 50),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': Integer(1, 5),
    'leaf_size': Integer(10, 50)
}


knn_model = KNeighborsClassifier(n_jobs = -1)

In [ ]:
np.random.seed(2025)
knn_grid = BayesSearchCV(knn_model, search_spaces = params, cv = split_num,  n_iter = 50, n_jobs = -1, scoring = 'f1_macro')
knn_grid.fit(X_temp_num, y_temp_num)
print('Best Parameters:', knn_grid.best_params_)

In [ ]:
np.random.seed(2025)

knn_model = KNeighborsClassifier(leaf_size = 50, metric = "minkowski", n_neighbors = 2, p = 5, weights = "distance", n_jobs = -1)
knn_model.fit(X_temp, y_temp)
y_knn_pred = knn_model.predict(X_test)

print(classification_report(y_test, y_knn_pred, zero_division = 1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [ ]:
np.random.seed(2025)
knn_model.fit(X_full, y)
y_knn = knn_model.predict(X_full_test)

In [ ]:
from google.colab import files

pred_knn = pd.DataFrame({'ID': range(1, 92), 'Placement': y_knn})

with open('KNN_Optimized_Normalized_Cov.csv', 'w') as file:
    pred_knn.to_csv(file, index = False, header = True)

files.download('KNN_Optimized_Normalized_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# XGBOOST

In [ ]:
params = {
    "n_estimators": Integer(50, 500),
    "eta": Real(0.01, 0.6, prior = "log-uniform"),
    "gamma": Real(0, 10, prior = "uniform"),
    "max_depth": Integer(3, 12),
    "min_child_weight": Integer(1, 10),
    "subsample": Real(0.5, 1.0, prior = "uniform"),
    "colsample_bytree": Real(0.5, 1.0, prior = "uniform"),
    "colsample_bylevel": Real(0.5, 1.0, prior = "uniform"),
    "colsample_bynode": Real(0.5, 1.0, prior = "uniform"),
    "reg_lambda": Real(1e-3, 10, prior = "log-uniform"),
    "alpha": Real(1e-3, 10, prior = "log-uniform"),
    "scale_pos_weight": Real(1, 10, prior = "uniform"),
    "max_delta_step": Integer(0, 10),
    "booster": Categorical(["gbtree", "gblinear", "dart"]),
    "grow_policy": Categorical(["depthwise", "lossguide"])
}

xgb_model = XGBClassifier(random_state = 2025, n_jobs = -1)

In [ ]:
np.random.seed(2025)
xgb_grid = BayesSearchCV(xgb_model, search_spaces = params, cv = split, n_iter = 50, n_jobs = -1, scoring = 'f1_macro')
xgb_grid.fit(X_temp, y_temp)
print('Best Parameters:', xgb_grid.best_params_)

In [ ]:
np.random.seed(2025)

xgb_model = XGBClassifier(
    alpha = 0.001,
    booster = 'gbtree',
    colsample_bylevel = 0.7374861995322094,
    colsample_bynode = 0.7374861995322094,
    colsample_bytree = 1.0,
    eta = 0.36299021453674973,
    gamma = 1.6878926002829784,
    grow_policy = 'lossguide',
    max_delta_step = 5,
    max_depth = 10,
    min_child_weight = 7,
    n_estimators = 500,
    reg_lambda = 2.0498535210377247,
    scale_pos_weight = 10.0,
    subsample = 0.5,
    random_state = 2025
)

xgb_model.fit(X_temp, y_temp)
y_xgb_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_xgb_pred, zero_division = 1))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86        36
           1       0.50      1.00      0.67         9

    accuracy                           0.80        45
   macro avg       0.75      0.88      0.76        45
weighted avg       0.90      0.80      0.82        45



In [ ]:
np.random.seed(2025)
xgb_model.fit(X_full, y)
y_xgb = xgb_model.predict(X_full_test)

In [ ]:
from google.colab import files

pred_xgb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_xgb})

with open('XGB_Optimized_Normalized_Cov.csv', 'w') as file:
    pred_xgb.to_csv(file, index = False, header = True)

files.download('XGB_Optimized_Normalized_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# MLP

In [ ]:
y_xgb_fit = xgb_model.predict(X_full)
y_knn_fit = knn_model.predict(X_full)
X_full_new = np.column_stack((X_full, y_xgb_fit, y_knn_fit))

y_xgb_pred = xgb_model.predict(X_full_test)
y_knn_pred = knn_model.predict(X_full_test)
X_full_test_new = np.column_stack((X_full_test, y_xgb_pred, y_knn_pred))

In [ ]:
np.random.seed(2025)
ensm_model = MLPClassifier(activation = 'relu',
                           alpha = 0.0004073523221358627,
                           learning_rate = 'constant',
                           learning_rate_init = 0.0007155492808659906,
                           max_iter = 947,
                           batch_size = 28,
                           early_stopping = True,
                           solver = 'adam',
                           random_state = 2025)

ensm_model.fit(X_full_new, y)
y_ensm_pred = ensm_model.predict(X_full_test_new)

In [ ]:
from google.colab import files

pred_ensm = pd.DataFrame({'ID': range(1, 92), 'Placement': y_ensm_pred})

with open('ENSM_Optimized_Normalized_Cov.csv', 'w') as file:
    pred_ensm.to_csv(file, index = False, header = True)

files.download('ENSM_Optimized_Normalized_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>